# EmailSearchAI

A generative search system for emails that helps organisations find and validate past decisions, strategies, and data in a huge corpus of email threads.

Steps:

1. Ingest emails and their metadata into a vector database.
2. Implement a retrieval-augmented generation (RAG) architecture.
3. Use embeddings to find relevant email threads.
4. Generate responses based on retrieved emails.


## The Embedding Layer

### About dataset
- The dataset is provided in CSV and Pickle formats for ease of use.
- Each thread contains multiple emails, allowing for analysis of conversation flow and decision-making processes.
- Human-generated summaries enable quick understanding and validation of thread content.
- Suitable for tasks such as search, summarization, and retrieval-augmented generation in enterprise settings.

**Email Thread Summary Dataset**

**Overview:**  
The Email Thread Dataset consists of two main files: `email_thread_details` and `email_thread_summaries`. These files collectively offer a comprehensive compilation of email thread information alongside human-generated summaries.

**Email Thread Details**  
*Description:*  
The `email_thread_details` file provides a detailed perspective on individual email threads, encompassing crucial information such as subject, timestamp, sender, recipients, and the content of the email.

*Columns:*
- `thread_id`: A unique identifier for each email thread.
- `subject`: Subject of the email thread.
- `timestamp`: Timestamp indicating when the message was sent.
- `from`: Sender of the email.
- `to`: List of recipients of the email.
- `body`: Content of the email message.

*Additional Information:*  
The "to" column is available in both CSV and Pickle (pkl) formats, facilitating convenient access to recipient information as a column of lists of strings.

**Email Thread Summaries**  
*Description:*  
The `email_thread_summaries` file contains concise summaries crafted by human annotators for each email thread, offering a high-level overview of the content.

*Columns:*
- `thread_id`: A unique identifier for each email thread.
- `summary`: A concise summary of the email thread.

**Dataset Structure:**  
The dataset is organized into threads and emails. There are a total of 4,167 threads and 21,684 emails, providing a rich source of information for analysis and research purposes.

- Threads: 4,167 threads  
- Emails: 21,684 emails

**Language:**  
- Languages: English (en)

In [270]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import chromadb
import openai
import tiktoken
from huggingface_hub import hf_hub_download
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
import re

In [271]:
email_thread_details = pd.read_csv("./dataset/csv/trimmed_email_thread_details.csv")
email_thread_summaries = pd.read_csv("./dataset/csv/trimmed_email_thread_summaries.csv")

In [272]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [273]:
email_thread_details.head()

,thread_id,subject,timestamp,from,to,body
0,1,FW: Master Termination Log,2002-01-29 11:23:42,"Gossett, Jeffrey C. JGOSSET","['Giron', 'Darron C. Dgiron', 'Love', 'Phillip M. Plove']","\n\n -----Original Message-----\nFrom: =09Theriot, Kim S. =20\nSent:=09Tuesday, January 29, 2002 1:23 PM\nTo:=09Richardson, Stacey; Anderson, Diane; Gossett, Jeffrey C.; White, Stac=\ney W.; Murphy, Melissa; Hall, D. Todd; Sweeney, Kevin\nCc:=09Aucoin, Evelyn; Baxter, Bryce; Wynne, Rita\nSubject:=09FW: Master Termination Log\n\n\n\n -----Original Message-----\nFrom: =09Panus, Stephanie =20\nSent:=09Tuesday, January 29, 2002 11:39 AM\nTo:=09Adams, Laurel; Alonso, Tom; Aronowitz, Alan; Bailey, Susan; Balfour-F=\nlanagan, Cyndie; Baughman, Edward; Belden, Tim; Bishop, Serena; Brackett, D=\nebbie R.; Bradford, William S.; Browning, Mary Nell; Bruce, James; Bruce, M=\nichelle; Bruce, Robert; Buerkle, Jim; Calger, Christopher F.; Carrington, C=\nlara; Considine, Keith; Cordova, Karen A.; Crandall, Sean; Cutsforth, Diane=\n; Diamond, Russell; Dunton, Heather; Edison, Susan; Elafandi, Mo; Fischer, =\nMark; Flores, Nony; Fondren, Mark; Gorny, Vladimir; Gorte, David; Gresham, =\nWayne; Hagelmann, Bjorn; Hall, Steve C. (Legal); Harkness, Cynthia; Hendry,=\n Brent; Johnston, Greg; Keohane, Peter; Lindeman, Cheryl; Little, Kelli; Ma=\nllory, Chris; Mann, Kay; Mcginnis, Stephanie; McGrory, Robert; McMichael Jr=\n., Ed; Miller, Don (Asset Mktg); Moore, Janet H.; Moran, Tom; Murphy, Harla=\nn; Murray, Julia; Nemec, Gerald; Ogden, Mary; Otto, Randy; Page, Jonalan; P=\nostlethwaite, John; Prejean, Frank; Presto, Kevin M.; Puchot, Paul; Rasmuss=\nen, Dale; Richter, Brad; Richter, Jeff; Robison, Michael A.; Rohauer, Tanya=\n; Rosman, Stewart; Runswick, Stacy; Sacks, Edward; Scholtes, Diana; Shackle=\nton, Sara; Simons, Paul; Swinney, John; Thapar, Raj; Theriot, Kim S.; Thoma=\ns, Jake; Thome, Stephen; Tricoli, Carl; Van Hooser, Steve; Wente, Laura; Wi=\nlson, Shona; Winfree, O'Neal D.; Woodland, Andrea; Yoder, Christian\nSubject:=09Master Termination Log\n\nAttached is the Daily Termination List for January 25 as well as the Master=\n Termination Log, which incorporates all terminations received through Janu=\nary 25.\n\n =20\n\nThe following were previously on the Master Termination Log and have now be=\nen marked as ""Y"" for a valid termination:\n\nAtlantic Coast Fibers, Inc.=09=09=09ENA=09=09pulp/paper transactions\nCNC-Containers Corporation=09=09=09EPMI=09=09master power agreement\nPublic Utility District No. 1 of Chelan County=09EPMI=09=09deal no. 757497.=\n01\nConnect Energy Services, Inc.=09=09=09ENA=09=09liquids agreement\nNGL Supply, Inc. (including Premier=09=09ENA/EGLI=09physical & financial tr=\nansactions referenced\nEnergy Partners, a division of NGL Supply, Inc.)\nPlains Marketing, L.P.=09=09=09=09ERAC=09=09deal no. QG4563.1\nPlains Marketing, L.P.=09=09=09=09ERAC=09=09deal no. QG4482.2\n\nStephanie Panus\nEnron Wholesale Services\nph: 713.345.3249\nfax: 713.646.3490"
1,1,FW: Master Termination Log,2002-01-31 12:50:00,"Theriot, Kim S. KTHERIO","['Murphy', 'Melissa Mmurphy', 'Gossett', 'Jeffrey C. Jgosset', 'White', 'Stacey W. Swhite', 'Hall', 'D. Todd Thall', 'Sweeney', 'Kevin Ksweene', 'Anderson', 'Diane Danders2', 'Hunter', 'Larry Joe Jhunte2']","\n\n -----Original Message-----\nFrom: =09Panus, Stephanie =20\nSent:=09Thursday, January 31, 2002 12:08 PM\nTo:=09Adams, Laurel; Albrecht, Kristin; Alonso, Tom; Aronowitz, Alan; Baile=\ny, Susan; Balfour-Flanagan, Cyndie; Baughman, Edward; Belden, Tim; Bishop, =\nSerena; Boyd, Samantha; Brackett, Debbie R.; Bradford, William S.; Browning=\n, Mary Nell; Bruce, James; Bruce, Michelle; Bruce, Robert; Buerkle, Jim; Ca=\nlger, Christopher F.; Carrington, Clara; Considine, Keith; Cordova, Karen A=\n.; Crandall, Sean; Cutsforth, Diane; Diamond, Russell; Dunton, Heather; Edi=\nson, Susan; Elafandi, Mo; Fischer, Mark; Flores, Nony; Fondren, Mark; Gorny=\n, Vladimir; Gorte, David; Gresham, Wayne; Hagelmann, Bjorn;

In [274]:
email_thread_summaries['text_length'] = email_thread_summaries['summary'].apply(lambda x: len(x.split(' ')))

In [275]:
# Count number of emails per thread
email_thread_counts = email_thread_details.groupby('thread_id').size()
email_thread_summaries['num_emails'] = email_thread_summaries['thread_id'].map(email_thread_counts)

In [276]:
# Use an LLM to generate a summary of the related emails and people involved in the conversations

import json

def generate_response(related_emails, existing_summary):
    response = openai.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes email threads and identifies key participants. Always respond in JSON format."},
            {"role": "user", "content": f"""
                Summarize the following emails and identify the key people involved in the conversation.
                You will be provided a list of emails. It will be a list of strings.
                You will also be provided with an existing summary which you can use as a reference

                Following are the emails you need to summarize and identify the key people involved in the conversation: \n\n{related_emails}.
                Here is the existing summary you can use as a reference: \n\n{existing_summary}
                
                Provide only the summary and key people in the following format (respond in JSON):
                {{
                    "summary": "The summary of the emails...",
                    "key_people": ["person1@example.com", "person2@example.com"]
                }}
            """}
        ],
        response_format={"type": "json_object"},
    )

    json_output = json.loads(response.choices[0].message.content)
    return json_output

In [277]:
email_thread_details[email_thread_details['thread_id'] == 2]

,thread_id,subject,timestamp,from,to,body
5,2,Credit Group Lunch,2000-01-12 05:26:00,Tana Jones,['Suzanne Adams'],I'll be there...
6,2,Credit Group Lunch,2000-02-15 01:08:00,Tana Jones,['Suzanne Adams'],I will attend.
7,2,Credit Group Lunch,2000-04-18 04:54:00,Carol St Clair,['Suzanne Adams'],"Suzanne:\nHere is the complete list of credit folks. Please send an e-mail to each of \nthem concerning the 5th. Please include the description that I have bolded. \nIn our group, you don't need to include Marie or Shari. Thanks.\n\nCarol\n---------------------- Forwarded by Carol St Clair/HOU/ECT on 04/18/2000 \n11:52 AM ---------------------------\n \n\n\nFrom: John Suttle \n 04/18/2000 11:47 AM\t\n\t\n\t\n\t \n\t\n\nTo: Carol St Clair/HOU/ECT@ECT\ncc: \nSubject: Re: Credit Group Lunch \n\nCarol,\n\nThree more have recently joined our group:\nEd Sacks\nBrad Schneider\nWendy LeBrocq\n\nJS\n\n\n\nCarol St Clair\n04/18/2000 11:43 AM\nTo: John Suttle/HOU/ECT@ECT\ncc: \nSubject: Credit Group Lunch\n\nJohn:\nSara and I would like to hold another lunch with your group on Friday, May \n5th to go through in detail how the ISDA and CSA Masters and Schedules work. \nCould you please take a look at this list and let me know of any additions or \ndeletions? Thanks.\n\nCarol\n\nBill Bradford\nDebbie Brackett\nTanya Rohauer\nRod Nelson\nRussell Diamond\nVeronica Espinoza\nTracy Ngo\nBrant Reves\nKevin Radous\nTom Moran\nChristopher Smith\nLesli Campbell\nCathy Tudon\nNidia Martinez\nMolly Harris\n\nThanks.\n\nCarol\n\n\n\n\n\n"
8,2,Credit Group Lunch,2000-04-18 06:13:00,Carol St Clair,['Suzanne Adams'],Suzanne:\nCould you please check the names of Cathy Tudon and Nidia Martinez? They \nneed to be included on this and I wasn't sure if who we sent it to covered \nthem. Seems like last time we had a problem sending it to them.\nCarol
9,2,Credit Group Lunch,2000-04-18 08:25:00,Mark Taylor,['Suzanne Adams'],"I will not be able to attend.\n\n\n\n\nSuzanne Adams\n04/18/2000 12:05 PM\nTo: Carol St Clair/HOU/ECT@ECT, Mark Taylor/HOU/ECT@ECT, Sara \nShackleton/HOU/ECT@ECT, Tana Jones/HOU/ECT@ECT, Susan Flynn/HOU/ECT@ECT, \nSusan Bailey/HOU/ECT@ECT, Tanya Rohauer/HOU/ECT@ECT, William S \nBradford/HOU/ECT@ECT, Debbie R Brackett/HOU/ECT@ECT, Russell \nDiamond/HOU/ECT@ECT, Veronica Espinoza/Corp/Enron@ENRON, Tracy \nNgo/HOU/ECT@ECT, Brant Reves/HOU/ECT@ECT, Rod Nelson/HOU/ECT@ECT, John \nSuttle/HOU/ECT@ECT, Tom Moran/HOU/ECT@ECT, Christopher Smith/HOU/ECT@ECT, \nLesli Campbell/HOU/ECT@ECT, Mary Tudon/HOU/ECT@ECT, Paul \nRadous/Corp/Enron@ENRON, Molly Harris/HOU/ECT@ECT, Nidia Mendoza/HOU/ECT@ECT, \nEdward Sacks/Corp/Enron@Enron, Brad Schneider/Corp/Enron@Enron, Wendi \nLeBrocq/Corp/Enron@Enron\ncc: \nSubject: Credit Group Lunch\n\nA lunch meeting has been scheduled for Friday, May 5, 2000, from 12:00 p.m. \nuntil 1:30 p.m. in 30C2 to go through in detail how the ISDA and CSA Masters \nand Schedules work.\n\nPlease reply as soon as possible if you are going to attend this lunch \nmeeting (for catering purposes). Thanks.\n\n"
10,2,Credit Group Lunch,2000-04-18 08:29:00,Sara Shackleton,['Kaye Ellis'],"Gosh, I guessed right!!!!\n\n\n\n\nKaye Ellis\n04/18/2000 01:51 PM\nTo: Sara Shackleton/HOU/ECT@ECT\ncc: \nSubject: Re: Credit Group Lunch \n\nJeff Sorenson would like the meeting on May 12 to be from 11:30a to 1p.\n\n"
11,2,Credit Group Lunch,2000-04-18 09:18:00,Carol St Clair,['Suzanne Adams'],yes. That's okay.\nCarol


In [278]:
email_thread_details[email_thread_details['thread_id'] == 2]['body'].tolist()

["I'll be there...",
 'I will attend.',
 "Suzanne:\nHere is the complete list of credit folks.  Please send an e-mail to each of \nthem concerning the 5th.  Please include the description that I have bolded.  \nIn our group, you don't need to include Marie or Shari. Thanks.\n\nCarol\n---------------------- Forwarded by Carol St Clair/HOU/ECT on 04/18/2000 \n11:52 AM ---------------------------\n   \n\n\nFrom:  John Suttle                                                            \n 04/18/2000 11:47 AM\t\n\t\n\t\n\t                           \n\t\n\nTo: Carol St Clair/HOU/ECT@ECT\ncc:  \nSubject: Re: Credit Group Lunch  \n\nCarol,\n\nThree more have recently joined our group:\nEd Sacks\nBrad Schneider\nWendy LeBrocq\n\nJS\n\n\n\nCarol St Clair\n04/18/2000 11:43 AM\nTo: John Suttle/HOU/ECT@ECT\ncc:  \nSubject: Credit Group Lunch\n\nJohn:\nSara and I would like to hold another lunch with your group on Friday, May \n5th to go through in detail how the ISDA and CSA Masters and Schedules wo

In [279]:
import json
chat_gpt_response = generate_response(email_thread_details[email_thread_details['thread_id'] == 2]['body'].tolist(), email_thread_summaries['summary'][1])
chat_gpt_response['summary']

'A lunch meeting is set for May 5th to review ISDA and CSA Masters and Schedules with the credit group. Carol coordinates the attendee list, asks Suzanne to ensure all intended participants are contacted, and specifically checks that Cathy Tudon and Nidia Martinez are included. John Suttle adds Ed Sacks, Brad Schneider, and Wendy LeBrocq to the invitees. Suzanne sends the formal invitation and requests RSVPs. Some members respond with their attendance or regrets. There is also a mention of a potential alternative meeting on May 12th, suggested by Jeff Sorenson via Kaye Ellis, but Carol agrees to the May 5th date.'

In [280]:
email_thread_summaries[email_thread_summaries['thread_id'] == 2]['summary']

1    A lunch meeting has been scheduled for May 5th from 12:00 p.m. to 1:30 p.m. to discuss the ISDA and CSA Masters and Schedules. Attendees are asked to RSVP for catering purposes. Carol requests confirmation of attendees and adds three new members to the group. John confirms the lunch and suggests two additional names to include. Suzanne is asked to send an email to all credit group members. Carol and Sara express their attendance.
Name: summary, dtype: object

In [281]:
# Add a generated summary and key_people as a column in email_thread_summary

# iterate over email_thread_summaries and create 2 new columns

# Generate summaries and key people for each thread
def get_summary_and_people(thread_id):
    bodies = email_thread_details[email_thread_details['thread_id'] == thread_id]['body'].tolist()
    existing_summary = email_thread_summaries[email_thread_summaries['thread_id'] == thread_id]['summary'].values[0]
    if bodies:
        result = generate_response(bodies, existing_summary)
        return result['summary'], ", ".join(result['key_people'])
    else:
        return "", ""

email_thread_summaries[['generated_summary', 'key_people']] = email_thread_summaries['thread_id'].apply(
    lambda x: pd.Series(get_summary_and_people(x))
)



In [282]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
email_thread_summaries.head(2)


,thread_id,summary,text_length,num_emails,generated_summary,key_people
0,1,The email thread discusses the Master Termination Log and the need to investigate a CNG LDC (Hope Gas) termination and a $66 million settlement offer. Stephanie Panus sends out the Daily List and Master Termination Log for various dates. Kim Theriot requests her name and Melissa Murphy's name to be removed from the distribution list and adds several names to it. The thread also includes updates on terminations and valid terminations for various companies.,74,5,"This email thread centers on the distribution and updates of the Enron Master Termination Log, which tracks contract terminations and relevant details (such as company names and deal numbers). Stephanie Panus is the primary sender providing the updated Daily Lists and Master Termination Logs covering January through May 2002. Notable updates include marking certain terminations as valid and the inclusion of prepetition mutual terminations. Kim Theriot requests the removal of her own and Melissa Murphy’s names from the distribution list and adds several new recipients. In May, Ed McMichael Jr. forwards a request to colleagues to investigate the CNG LDC (Hope Gas) termination and a related $66 million settlement offer listed in the logs. Multiple Enron personnel are either recipients or mentioned in these communications, reflecting the broad organizational reach of the terminated contracts tracking process.","Stephanie Panus, Kim Theriot, Ed McMichael Jr., Melissa Murphy, Todd D. Hall, Kevin Sweeney, Rita Wynne, Rebecca Grace, Rhonda Robinson, Kerri Thompson, Kristin Albrecht, Tom Chapman, Stacey Richardson, Diane Anderson, Jeffrey C. Gossett, Stacey W. White, D. Todd Hall, Bryce Baxter, Evelyn Aucoin, Rita Wynne, Fred Lagrasta, Katherine L. Kelly, Victoria Versen"
1,2,A lunch meeting has been scheduled for May 5th from 12:00 p.m. to 1:30 p.m. to discuss the ISDA and CSA Masters and Schedules. Attendees are asked to RSVP for catering purposes. Carol requests confirmation of attendees and adds three new members to the group. John confirms the lunch and suggests two additional names to include. Suzanne is asked to send an email to all credit group members. Carol and Sara express their attendance.,74,7,"The email thread discusses organizing a Credit Group Lunch on May 5th to review the ISDA and CSA Masters and Schedules. Carol coordinates the attendee list and communication, asking Suzanne to ensure all credit group members, specifically Cathy Tudon and Nidia Martinez, are included, as they were missed previously. John informs Carol about three new group members (Ed Sacks, Brad Schneider, Wendy LeBrocq). Suzanne sends out the invitation and manages RSVPs, while various recipients confirm or decline attendance. There is also some discussion about scheduling a subsequent meeting for May 12th.","Carol St Clair, Suzanne Adams, John Suttle, Sara Shackleton, Kaye Ellis, Ed Sacks, Brad Schneider, Wendy LeBrocq, Cathy Tudon, Nidia Martinez"


In [283]:
email_thread_summaries.info

<bound method DataFrame.info of    thread_id  \
0          1   
1          2   
2          3   
3          4   
4          5   
5          6   
6          7   
7          8   
8          9   
9         10   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      summary  \
0                                                                                     

In [284]:
# Stop at this cell

# raise RuntimeError("This is a purposeful error for demonstration.")



- We will create 2 tables in chroma db
- First match query with email_thread_summaries
- Then find top k documents from 2nd table in email_thread_details

In [285]:
chroma_client = chromadb.Client()

In [286]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [287]:
chroma_data_path = './chroma_data'

In [288]:
import chromadb

In [289]:
# Call PersistentClient()

client = chromadb.PersistentClient(chroma_data_path)


In [290]:
# Set up the embedding function using the OpenAI embedding model
model = "text-embedding-ada-002"
embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key, model_name=model)

In [291]:
# Initialise a collection in chroma and pass the embedding_function to it so that it used OpenAI embeddings to embed the documents

email_summaries_collection = client.get_or_create_collection(name='Email_Summaries', embedding_function=embedding_function)

In [292]:
# Batch the data to avoid exceeding the token limit per request
batch_size = 100  # You can adjust this value if needed

summaries = email_thread_summaries['generated_summary'].tolist()
metadatas = email_thread_summaries[['thread_id', 'key_people', 'num_emails', 'text_length']].to_dict(orient='records')
ids = [str(i) for i in range(len(email_thread_summaries))]

for start_idx in range(0, len(summaries), batch_size):
    end_idx = start_idx + batch_size
    email_summaries_collection.add(
        documents=summaries[start_idx:end_idx],
        metadatas=metadatas[start_idx:end_idx],
        ids=ids[start_idx:end_idx],
    )

In [293]:
email_details_collection = client.get_or_create_collection(name='Email_Details', embedding_function=embedding_function)


In [294]:
# Batch the data to avoid exceeding the token limit per request
details_batch_size = 100  # You can adjust this value if needed

details_documents = email_thread_details['body'].tolist()
details_metadatas = email_thread_details[['thread_id', 'subject', 'from', 'to', 'timestamp']].to_dict(orient='records')
details_ids = [str(i) for i in range(len(email_thread_details))]

# Estimate tokens using tiktoken
encoding = tiktoken.encoding_for_model(model)
max_tokens = 8192

def count_tokens(text):
    return len(encoding.encode(text))

current_batch_docs = []
current_batch_metas = []
current_batch_ids = []
current_tokens = 0

for doc, meta, doc_id in zip(details_documents, details_metadatas, details_ids):
    doc_tokens = count_tokens(doc)
    # Skip documents that exceed the model's max token limit
    if doc_tokens > max_tokens:
        continue
    # If adding this document exceeds the token limit, flush the batch
    if current_tokens + doc_tokens > max_tokens and current_batch_docs:
        email_details_collection.add(
            documents=current_batch_docs,
            metadatas=current_batch_metas,
            ids=current_batch_ids,
        )
        current_batch_docs = []
        current_batch_metas = []
        current_batch_ids = []
        current_tokens = 0
    current_batch_docs.append(doc)
    current_batch_metas.append(meta)
    current_batch_ids.append(doc_id)
    current_tokens += doc_tokens

# Add any remaining documents in the last batch
if current_batch_docs:
    email_details_collection.add(
        documents=current_batch_docs,
        metadatas=current_batch_metas,
        ids=current_batch_ids,
    )


In [295]:
email_details_collection.peek()

{'ids': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 'embeddings': array([[-0.00499778, -0.00385044, -0.01661965, ..., -0.01360464,
         -0.01789694, -0.03531861],
        [-0.01089431,  0.00983236, -0.00881496, ..., -0.01803094,
         -0.0290515 , -0.03190318],
        [-0.01914887,  0.00240259, -0.02326041, ..., -0.0100129 ,
         -0.02975837, -0.01784065],
        ...,
        [-0.0215793 ,  0.00223012,  0.01240775, ..., -0.01608334,
         -0.00317517, -0.02149561],
        [-0.0128343 ,  0.00501978,  0.00415858, ...,  0.00800887,
         -0.02688835, -0.00522419],
        [-0.02208114, -0.01466031, -0.0001214 , ..., -0.01145904,
         -0.00528315, -0.02715854]], shape=(10, 1536)),
 'documents': ['\n\n -----Original Message-----\nFrom: =09Theriot, Kim S. =20\nSent:=09Tuesday, January 29, 2002 1:23 PM\nTo:=09Richardson, Stacey; Anderson, Diane; Gossett, Jeffrey C.; White, Stac=\ney W.; Murphy, Melissa; Hall, D. Todd; Sweeney, Kevin\nCc:=09Aucoin, Evelyn; Baxt

## The Search Layer

In [296]:
# query = "Who proposed the chosen approach for the data migration and when?"
query = input()

In [297]:
related_threads = email_summaries_collection.query(
    query_texts=query,
    n_results=10
)

In [298]:
related_threads.items()

# convert into data frame

# Convert related_threads to a DataFrame
summary_df = pd.DataFrame([
    {
        'thread_id': meta['thread_id'],
        'key_people': meta['key_people'],
        'num_emails': meta['num_emails'],
        'text_length': meta['text_length'],
        'summary': doc
    }
    for meta, doc in zip(related_threads['metadatas'][0], related_threads['documents'][0])
])

In [299]:
summary_df.head()

,thread_id,key_people,num_emails,text_length,summary
0,2,"Carol St Clair, Sara Shackleton, Suzanne Adams, John Suttle, Kaye Ellis, Jeff Sorenson",7,74,"A Credit Group lunch is scheduled for May 5th to review ISDA and CSA documentation. Carol St Clair and Sara Shackleton are coordinating the event and confirming the participants, with Carol ensuring the inclusion of all relevant credit group members, specifically flagging Cathy Tudon and Nidia Martinez for verification. John Suttle provides additional names to be invited (Ed Sacks, Brad Schneider, Wendi LeBrocq). Suzanne Adams sends out the formal invitation and collects RSVPs for catering. Additional scheduling matters are raised for a different meeting on May 12th. Attendees are asked to confirm participation."
1,9,"erwollam@hotmail.com, Joe.Parks@bridgeline.net, Ben, Jay Brown, John Brown",5,86,"Ben updates the group about his upcoming NYC Marathon participation, proposes a friendly wager, and shares his busy schedule, including job offers from investment banks and recent travels. Erik asks Joe about renting a trailer for $45 from Aztec and inquires about getting a better price. Joe apologizes for his delay in responding, citing a busy period. The conversation includes lighthearted comments and references to mutual acquaintances, with all participants catching up on recent events and future plans."
2,8,"Jeff.Dasovich@ENRON.com, Nancy.Sellers@RobertMondavi.com, scottwl@hotmail.com, eldon@direcpc.com, psellers@pacbell.net, cameron@perfect.com",5,71,"The group discusses coordinating a golf outing in Napa during the weekend. Jeff Dasovich initiates the idea, but scheduling conflicts arise due to Eldon's concert rehearsals and Scott's mandolin lesson. Nancy suggests alternative courses since Eldon is unavailable at NVCC. Scott and Cameron consider Saturday afternoon or Sunday but ultimately decide to stay in San Francisco due to a busy travel schedule. There is also a brief mention of mailing 'stuff' and confirming receipt of a check, but golf logistics are the main focus. Plans to play are ultimately canceled due to scheduling complications."
3,1,"Stephanie Panus, Kim S. Theriot, Melissa Murphy, Todd D. Hall, Kevin Sweeney, Rita Wynne, Rebecca Grace, Rhonda Robinson, Kerri Thompson, Kristin Albrecht, Tom Chapman, Ed McMichael Jr., Fred Lagrasta, Katherine L. Kelly, Victoria Versen",5,74,"The emails center around the distribution and updates of the Master Termination Log, which tracks contract terminations and settlements at Enron. Stephanie Panus regularly sends out updates and attachments regarding terminations, including new daily lists and additions such as prepetition mutual terminations. Kim Theriot requests changes to the distribution list, removing herself and Melissa Murphy and adding other colleagues to ensure the right parties are informed. In a later exchange, Ed McMichael Jr. requests a review of a specific CNG LDC (Hope Gas) termination and a $66 million settlement offer, involving Fred Lagrasta, Katherine Kelly, and Victoria Versen in the investigation. The conversation demonstrates ongoing management and review of entity terminations, distribution list housekeeping, and specific termination inquiry follow-ups."
4,3,"Bruce Rudy, Brian Hendon, Gerald Nemec, Kristy Carnes, Janel Guerrero, Robin Rodrigue, Gwendolyn Gray",6,55,"The majority of this email thread centers around Bruce Rudy notifying colleagues of his move to Denver and sharing his new address and contact information. A playful back-and-forth ensues between Bruce Rudy, Brian Hendon, and Gerald Nemec regarding the fate of Bruce's furniture, with jokes about ownership and legal rights, referencing office friendships and Texas law humorously. In a separate exchange, Robin Rodrigue provides Gwendolyn Gray with her new address in Chicago so that records can be updated. Overall, the correspondence consists of relocation updates and friendly banter among coworkers."


In [300]:
# Let's look at the related emails based on the fetched summary entries

related_emails = email_details_collection.query(
    query_texts = [query],
    n_results=10,
    where={"thread_id": {"$in": summary_df['thread_id'].tolist()}}
)


In [301]:
related_emails

{'ids': [['7', '10', '9', '39', '6', '8', '45', '38', '23', '2']],
 'embeddings': None,
 'documents': [["Suzanne:\nHere is the complete list of credit folks.  Please send an e-mail to each of \nthem concerning the 5th.  Please include the description that I have bolded.  \nIn our group, you don't need to include Marie or Shari. Thanks.\n\nCarol\n---------------------- Forwarded by Carol St Clair/HOU/ECT on 04/18/2000 \n11:52 AM ---------------------------\n   \n\n\nFrom:  John Suttle                                                            \n 04/18/2000 11:47 AM\t\n\t\n\t\n\t                           \n\t\n\nTo: Carol St Clair/HOU/ECT@ECT\ncc:  \nSubject: Re: Credit Group Lunch  \n\nCarol,\n\nThree more have recently joined our group:\nEd Sacks\nBrad Schneider\nWendy LeBrocq\n\nJS\n\n\n\nCarol St Clair\n04/18/2000 11:43 AM\nTo: John Suttle/HOU/ECT@ECT\ncc:  \nSubject: Credit Group Lunch\n\nJohn:\nSara and I would like to hold another lunch with your group on Friday, May \n5th to go 

In [302]:
# convert related_emails into a dataframe
related_emails_df = pd.DataFrame({
    'thread_id': [meta['thread_id'] for meta in related_emails['metadatas'][0]],
    'subject': [meta['subject'] for meta in related_emails['metadatas'][0]],
    'from': [meta['from'] for meta in related_emails['metadatas'][0]],
    'to': [meta['to'] for meta in related_emails['metadatas'][0]],
    'timestamp': [meta['timestamp'] for meta in related_emails['metadatas'][0]],
    'body': [doc for doc in related_emails['documents'][0]],
    'distance': [dist for dist in related_emails['distances'][0]]
})

In [303]:
related_emails_df.head()

,thread_id,subject,from,to,timestamp,body,distance
0,2,Credit Group Lunch,Carol St Clair,['Suzanne Adams'],2000-04-18 04:54:00,"Suzanne:\nHere is the complete list of credit folks. Please send an e-mail to each of \nthem concerning the 5th. Please include the description that I have bolded. \nIn our group, you don't need to include Marie or Shari. Thanks.\n\nCarol\n---------------------- Forwarded by Carol St Clair/HOU/ECT on 04/18/2000 \n11:52 AM ---------------------------\n \n\n\nFrom: John Suttle \n 04/18/2000 11:47 AM\t\n\t\n\t\n\t \n\t\n\nTo: Carol St Clair/HOU/ECT@ECT\ncc: \nSubject: Re: Credit Group Lunch \n\nCarol,\n\nThree more have recently joined our group:\nEd Sacks\nBrad Schneider\nWendy LeBrocq\n\nJS\n\n\n\nCarol St Clair\n04/18/2000 11:43 AM\nTo: John Suttle/HOU/ECT@ECT\ncc: \nSubject: Credit Group Lunch\n\nJohn:\nSara and I would like to hold another lunch with your group on Friday, May \n5th to go through in detail how the ISDA and CSA Masters and Schedules work. \nCould you please take a look at this list and let me know of any additions or \ndeletions? Thanks.\n\nCarol\n\nBill Bradford\nDebbie Brackett\nTanya Rohauer\nRod Nelson\nRussell Diamond\nVeronica Espinoza\nTracy Ngo\nBrant Reves\nKevin Radous\nTom Moran\nChristopher Smith\nLesli Campbell\nCathy Tudon\nNidia Martinez\nMolly Harris\n\nThanks.\n\nCarol\n\n\n\n\n\n",0.311754
1,2,Credit Group Lunch,Sara Shackleton,['Kaye Ellis'],2000-04-18 08:29:00,"Gosh, I guessed right!!!!\n\n\n\n\nKaye Ellis\n04/18/2000 01:51 PM\nTo: Sara Shackleton/HOU/ECT@ECT\ncc: \nSubject: Re: Credit Group Lunch \n\nJeff Sorenson would like the meeting on May 12 to be from 11:30a to 1p.\n\n",0.331361
2,2,Credit Group Lunch,Mark Taylor,['Suzanne Adams'],2000-04-18 08:25:00,"I will not be able to attend.\n\n\n\n\nSuzanne Adams\n04/18/2000 12:05 PM\nTo: Carol St Clair/HOU/ECT@ECT, Mark Taylor/HOU/ECT@ECT, Sara \nShackleton/HOU/ECT@ECT, Tana Jones/HOU/ECT@ECT, Susan Flynn/HOU/ECT@ECT, \nSusan Bailey/HOU/ECT@ECT, Tanya Rohauer/HOU/ECT@ECT, William S \nBradford/HOU/ECT@ECT, Debbie R Brackett/HOU/ECT@ECT, Russell \nDiamond/HOU/ECT@ECT, Veronica Espinoza/Corp/Enron@ENRON, Tracy \nNgo/HOU/ECT@ECT, Brant Reves/HOU/ECT@ECT, Rod Nelson/HOU/ECT@ECT, John \nSuttle/HOU/ECT@ECT, Tom Moran/HOU/ECT@ECT, Christopher Smith/HOU/ECT@ECT, \nLesli Campbell/HOU/ECT@ECT, Mary Tudon/HOU/ECT@ECT, Paul \nRadous/Corp/Enron@ENRON, Molly Harris/HOU/ECT@ECT, Nidia Mendoza/HOU/ECT@ECT, \nEdward Sacks/Corp/Enron@Enron, Brad Schneider/Corp/Enron@Enron, Wendi \nLeBrocq/Corp/Enron@Enron\ncc: \nSubject: Credit Group Lunch\n\nA lunch meeting has been scheduled for Friday, May 5, 2000, from 12:00 p.m. \nuntil 1:30 p.m. in 30C2 to go through in detail how the ISDA and CSA Masters \nand Schedules work.\n\nPlease reply as soon as possible if you are going to attend this lunch \nmeeting (for catering purposes). Thanks.\n\n",0.358281
3,8,RE: Golf Anyone?,"Dasovich, Jeff JDASOVIC","[""'Scott Laughlin' <scottwl@hotmail.com""]",2001-10-17 12:05:16,"Cool. If $150 don't cut it on the check, just let me know. No sweat. EVERYTHING is confidential.\n\n-----Original Message-----\nFrom: Scott Laughlin [mailto:scottwl@hotmail.com]\nSent: Wednesday, October 17, 2001 1:29 PM\nTo: Dasovich, Jeff\nSubject: Re: Golf Anyone?\n\n\nYOur e-mails are like icebergs. Four lines of text, and then a whole mound \nof explanation underneath.\n\nCameron and I are talking about the golf. It's looking good, but I'm not yet \nsure what day. I'll e-mail you later with the final verdict.\n\nAnd I got the check. Cheers.\n\n\n>From: ""Dasovich, Jeff"" <Jeff.Dasovich@ENRON.com>\n>To: <psellers@pacbell.net>, ""Nancy Sellers (E-mail)"" \n><Nancy.Sellers@RobertMondavi.com>, ""eldon sellers (E-mail)"" \n><eldon@direcpc.com>, ""Scott Laughlin (E-mail)"" <scottwl@hotmail.com>, \n><cameron@perfect.com>\n>Subject: Golf Anyone?\n>Date: Wed, 17 Oct 2001 10:26:52 -0500\n>\n>Looks like Prentice and Nancy will be getting together in Napa to do a\n>little bonding this weeke

In [304]:
from sentence_transformers.cross_encoder import CrossEncoder

model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")
scores = model.predict([["My first", "sentence pair"], ["Second text", "pair"]])
scores

array([-11.01487 , -10.854008], dtype=float32)

In [305]:
cross_inputs = [[query, response] for response in related_emails_df['body']]
cross_rerank_scores = model.predict(cross_inputs)

In [306]:
related_emails_df['ranking'] = cross_rerank_scores

In [307]:
related_emails_df.head()

,thread_id,subject,from,to,timestamp,body,distance,ranking
0,2,Credit Group Lunch,Carol St Clair,['Suzanne Adams'],2000-04-18 04:54:00,"Suzanne:\nHere is the complete list of credit folks. Please send an e-mail to each of \nthem concerning the 5th. Please include the description that I have bolded. \nIn our group, you don't need to include Marie or Shari. Thanks.\n\nCarol\n---------------------- Forwarded by Carol St Clair/HOU/ECT on 04/18/2000 \n11:52 AM ---------------------------\n \n\n\nFrom: John Suttle \n 04/18/2000 11:47 AM\t\n\t\n\t\n\t \n\t\n\nTo: Carol St Clair/HOU/ECT@ECT\ncc: \nSubject: Re: Credit Group Lunch \n\nCarol,\n\nThree more have recently joined our group:\nEd Sacks\nBrad Schneider\nWendy LeBrocq\n\nJS\n\n\n\nCarol St Clair\n04/18/2000 11:43 AM\nTo: John Suttle/HOU/ECT@ECT\ncc: \nSubject: Credit Group Lunch\n\nJohn:\nSara and I would like to hold another lunch with your group on Friday, May \n5th to go through in detail how the ISDA and CSA Masters and Schedules work. \nCould you please take a look at this list and let me know of any additions or \ndeletions? Thanks.\n\nCarol\n\nBill Bradford\nDebbie Brackett\nTanya Rohauer\nRod Nelson\nRussell Diamond\nVeronica Espinoza\nTracy Ngo\nBrant Reves\nKevin Radous\nTom Moran\nChristopher Smith\nLesli Campbell\nCathy Tudon\nNidia Martinez\nMolly Harris\n\nThanks.\n\nCarol\n\n\n\n\n\n",0.311754,1.597221
1,2,Credit Group Lunch,Sara Shackleton,['Kaye Ellis'],2000-04-18 08:29:00,"Gosh, I guessed right!!!!\n\n\n\n\nKaye Ellis\n04/18/2000 01:51 PM\nTo: Sara Shackleton/HOU/ECT@ECT\ncc: \nSubject: Re: Credit Group Lunch \n\nJeff Sorenson would like the meeting on May 12 to be from 11:30a to 1p.\n\n",0.331361,1.810978
2,2,Credit Group Lunch,Mark Taylor,['Suzanne Adams'],2000-04-18 08:25:00,"I will not be able to attend.\n\n\n\n\nSuzanne Adams\n04/18/2000 12:05 PM\nTo: Carol St Clair/HOU/ECT@ECT, Mark Taylor/HOU/ECT@ECT, Sara \nShackleton/HOU/ECT@ECT, Tana Jones/HOU/ECT@ECT, Susan Flynn/HOU/ECT@ECT, \nSusan Bailey/HOU/ECT@ECT, Tanya Rohauer/HOU/ECT@ECT, William S \nBradford/HOU/ECT@ECT, Debbie R Brackett/HOU/ECT@ECT, Russell \nDiamond/HOU/ECT@ECT, Veronica Espinoza/Corp/Enron@ENRON, Tracy \nNgo/HOU/ECT@ECT, Brant Reves/HOU/ECT@ECT, Rod Nelson/HOU/ECT@ECT, John \nSuttle/HOU/ECT@ECT, Tom Moran/HOU/ECT@ECT, Christopher Smith/HOU/ECT@ECT, \nLesli Campbell/HOU/ECT@ECT, Mary Tudon/HOU/ECT@ECT, Paul \nRadous/Corp/Enron@ENRON, Molly Harris/HOU/ECT@ECT, Nidia Mendoza/HOU/ECT@ECT, \nEdward Sacks/Corp/Enron@Enron, Brad Schneider/Corp/Enron@Enron, Wendi \nLeBrocq/Corp/Enron@Enron\ncc: \nSubject: Credit Group Lunch\n\nA lunch meeting has been scheduled for Friday, May 5, 2000, from 12:00 p.m. \nuntil 1:30 p.m. in 30C2 to go through in detail how the ISDA and CSA Masters \nand Schedules work.\n\nPlease reply as soon as possible if you are going to attend this lunch \nmeeting (for catering purposes). Thanks.\n\n",0.358281,-3.020647
3,8,RE: Golf Anyone?,"Dasovich, Jeff JDASOVIC","[""'Scott Laughlin' <scottwl@hotmail.com""]",2001-10-17 12:05:16,"Cool. If $150 don't cut it on the check, just let me know. No sweat. EVERYTHING is confidential.\n\n-----Original Message-----\nFrom: Scott Laughlin [mailto:scottwl@hotmail.com]\nSent: Wednesday, October 17, 2001 1:29 PM\nTo: Dasovich, Jeff\nSubject: Re: Golf Anyone?\n\n\nYOur e-mails are like icebergs. Four lines of text, and then a whole mound \nof explanation underneath.\n\nCameron and I are talking about the golf. It's looking good, but I'm not yet \nsure what day. I'll e-mail you later with the final verdict.\n\nAnd I got the check. Cheers.\n\n\n>From: ""Dasovich, Jeff"" <Jeff.Dasovich@ENRON.com>\n>To: <psellers@pacbell.net>, ""Nancy Sellers (E-mail)"" \n><Nancy.Sellers@RobertMondavi.com>, ""eldon sellers (E-mail)"" \n><eldon@direcpc.com>, ""Scott Laughlin (E-mail)"" <scottwl@hotmail.com>, \n><cameron@perfect.com>\n>Subject: Golf Anyone?\n>Date: Wed, 17 Oct 2001 10:26:52 -0500\n>\n>Looks like Prentice and Nancy will be getting together in Napa

In [308]:
top_5_emails_by_distance = related_emails_df.sort_values(by='distance').head(5)

top_5_emails_by_distance

,thread_id,subject,from,to,timestamp,body,distance,ranking
0,2,Credit Group Lunch,Carol St Clair,['Suzanne Adams'],2000-04-18 04:54:00,"Suzanne:\nHere is the complete list of credit folks. Please send an e-mail to each of \nthem concerning the 5th. Please include the description that I have bolded. \nIn our group, you don't need to include Marie or Shari. Thanks.\n\nCarol\n---------------------- Forwarded by Carol St Clair/HOU/ECT on 04/18/2000 \n11:52 AM ---------------------------\n \n\n\nFrom: John Suttle \n 04/18/2000 11:47 AM\t\n\t\n\t\n\t \n\t\n\nTo: Carol St Clair/HOU/ECT@ECT\ncc: \nSubject: Re: Credit Group Lunch \n\nCarol,\n\nThree more have recently joined our group:\nEd Sacks\nBrad Schneider\nWendy LeBrocq\n\nJS\n\n\n\nCarol St Clair\n04/18/2000 11:43 AM\nTo: John Suttle/HOU/ECT@ECT\ncc: \nSubject: Credit Group Lunch\n\nJohn:\nSara and I would like to hold another lunch with your group on Friday, May \n5th to go through in detail how the ISDA and CSA Masters and Schedules work. \nCould you please take a look at this list and let me know of any additions or \ndeletions? Thanks.\n\nCarol\n\nBill Bradford\nDebbie Brackett\nTanya Rohauer\nRod Nelson\nRussell Diamond\nVeronica Espinoza\nTracy Ngo\nBrant Reves\nKevin Radous\nTom Moran\nChristopher Smith\nLesli Campbell\nCathy Tudon\nNidia Martinez\nMolly Harris\n\nThanks.\n\nCarol\n\n\n\n\n\n",0.311754,1.597221
1,2,Credit Group Lunch,Sara Shackleton,['Kaye Ellis'],2000-04-18 08:29:00,"Gosh, I guessed right!!!!\n\n\n\n\nKaye Ellis\n04/18/2000 01:51 PM\nTo: Sara Shackleton/HOU/ECT@ECT\ncc: \nSubject: Re: Credit Group Lunch \n\nJeff Sorenson would like the meeting on May 12 to be from 11:30a to 1p.\n\n",0.331361,1.810978
2,2,Credit Group Lunch,Mark Taylor,['Suzanne Adams'],2000-04-18 08:25:00,"I will not be able to attend.\n\n\n\n\nSuzanne Adams\n04/18/2000 12:05 PM\nTo: Carol St Clair/HOU/ECT@ECT, Mark Taylor/HOU/ECT@ECT, Sara \nShackleton/HOU/ECT@ECT, Tana Jones/HOU/ECT@ECT, Susan Flynn/HOU/ECT@ECT, \nSusan Bailey/HOU/ECT@ECT, Tanya Rohauer/HOU/ECT@ECT, William S \nBradford/HOU/ECT@ECT, Debbie R Brackett/HOU/ECT@ECT, Russell \nDiamond/HOU/ECT@ECT, Veronica Espinoza/Corp/Enron@ENRON, Tracy \nNgo/HOU/ECT@ECT, Brant Reves/HOU/ECT@ECT, Rod Nelson/HOU/ECT@ECT, John \nSuttle/HOU/ECT@ECT, Tom Moran/HOU/ECT@ECT, Christopher Smith/HOU/ECT@ECT, \nLesli Campbell/HOU/ECT@ECT, Mary Tudon/HOU/ECT@ECT, Paul \nRadous/Corp/Enron@ENRON, Molly Harris/HOU/ECT@ECT, Nidia Mendoza/HOU/ECT@ECT, \nEdward Sacks/Corp/Enron@Enron, Brad Schneider/Corp/Enron@Enron, Wendi \nLeBrocq/Corp/Enron@Enron\ncc: \nSubject: Credit Group Lunch\n\nA lunch meeting has been scheduled for Friday, May 5, 2000, from 12:00 p.m. \nuntil 1:30 p.m. in 30C2 to go through in detail how the ISDA and CSA Masters \nand Schedules work.\n\nPlease reply as soon as possible if you are going to attend this lunch \nmeeting (for catering purposes). Thanks.\n\n",0.358281,-3.020647
3,8,RE: Golf Anyone?,"Dasovich, Jeff JDASOVIC","[""'Scott Laughlin' <scottwl@hotmail.com""]",2001-10-17 12:05:16,"Cool. If $150 don't cut it on the check, just let me know. No sweat. EVERYTHING is confidential.\n\n-----Original Message-----\nFrom: Scott Laughlin [mailto:scottwl@hotmail.com]\nSent: Wednesday, October 17, 2001 1:29 PM\nTo: Dasovich, Jeff\nSubject: Re: Golf Anyone?\n\n\nYOur e-mails are like icebergs. Four lines of text, and then a whole mound \nof explanation underneath.\n\nCameron and I are talking about the golf. It's looking good, but I'm not yet \nsure what day. I'll e-mail you later with the final verdict.\n\nAnd I got the check. Cheers.\n\n\n>From: ""Dasovich, Jeff"" <Jeff.Dasovich@ENRON.com>\n>To: <psellers@pacbell.net>, ""Nancy Sellers (E-mail)"" \n><Nancy.Sellers@RobertMondavi.com>, ""eldon sellers (E-mail)"" \n><eldon@direcpc.com>, ""Scott Laughlin (E-mail)"" <scottwl@hotmail.com>, \n><cameron@perfect.com>\n>Subject: Golf Anyone?\n>Date: Wed, 17 Oct 2001 10:26:52 -0500\n>\n>Looks like Prentice and Nancy will be getting together in Napa

In [309]:
top_5_emails_by_ranking = related_emails_df.sort_values(by='ranking', ascending=False).head(5)

top_5_emails_by_ranking

,thread_id,subject,from,to,timestamp,body,distance,ranking
1,2,Credit Group Lunch,Sara Shackleton,['Kaye Ellis'],2000-04-18 08:29:00,"Gosh, I guessed right!!!!\n\n\n\n\nKaye Ellis\n04/18/2000 01:51 PM\nTo: Sara Shackleton/HOU/ECT@ECT\ncc: \nSubject: Re: Credit Group Lunch \n\nJeff Sorenson would like the meeting on May 12 to be from 11:30a to 1p.\n\n",0.331361,1.810978
0,2,Credit Group Lunch,Carol St Clair,['Suzanne Adams'],2000-04-18 04:54:00,"Suzanne:\nHere is the complete list of credit folks. Please send an e-mail to each of \nthem concerning the 5th. Please include the description that I have bolded. \nIn our group, you don't need to include Marie or Shari. Thanks.\n\nCarol\n---------------------- Forwarded by Carol St Clair/HOU/ECT on 04/18/2000 \n11:52 AM ---------------------------\n \n\n\nFrom: John Suttle \n 04/18/2000 11:47 AM\t\n\t\n\t\n\t \n\t\n\nTo: Carol St Clair/HOU/ECT@ECT\ncc: \nSubject: Re: Credit Group Lunch \n\nCarol,\n\nThree more have recently joined our group:\nEd Sacks\nBrad Schneider\nWendy LeBrocq\n\nJS\n\n\n\nCarol St Clair\n04/18/2000 11:43 AM\nTo: John Suttle/HOU/ECT@ECT\ncc: \nSubject: Credit Group Lunch\n\nJohn:\nSara and I would like to hold another lunch with your group on Friday, May \n5th to go through in detail how the ISDA and CSA Masters and Schedules work. \nCould you please take a look at this list and let me know of any additions or \ndeletions? Thanks.\n\nCarol\n\nBill Bradford\nDebbie Brackett\nTanya Rohauer\nRod Nelson\nRussell Diamond\nVeronica Espinoza\nTracy Ngo\nBrant Reves\nKevin Radous\nTom Moran\nChristopher Smith\nLesli Campbell\nCathy Tudon\nNidia Martinez\nMolly Harris\n\nThanks.\n\nCarol\n\n\n\n\n\n",0.311754,1.597221
2,2,Credit Group Lunch,Mark Taylor,['Suzanne Adams'],2000-04-18 08:25:00,"I will not be able to attend.\n\n\n\n\nSuzanne Adams\n04/18/2000 12:05 PM\nTo: Carol St Clair/HOU/ECT@ECT, Mark Taylor/HOU/ECT@ECT, Sara \nShackleton/HOU/ECT@ECT, Tana Jones/HOU/ECT@ECT, Susan Flynn/HOU/ECT@ECT, \nSusan Bailey/HOU/ECT@ECT, Tanya Rohauer/HOU/ECT@ECT, William S \nBradford/HOU/ECT@ECT, Debbie R Brackett/HOU/ECT@ECT, Russell \nDiamond/HOU/ECT@ECT, Veronica Espinoza/Corp/Enron@ENRON, Tracy \nNgo/HOU/ECT@ECT, Brant Reves/HOU/ECT@ECT, Rod Nelson/HOU/ECT@ECT, John \nSuttle/HOU/ECT@ECT, Tom Moran/HOU/ECT@ECT, Christopher Smith/HOU/ECT@ECT, \nLesli Campbell/HOU/ECT@ECT, Mary Tudon/HOU/ECT@ECT, Paul \nRadous/Corp/Enron@ENRON, Molly Harris/HOU/ECT@ECT, Nidia Mendoza/HOU/ECT@ECT, \nEdward Sacks/Corp/Enron@Enron, Brad Schneider/Corp/Enron@Enron, Wendi \nLeBrocq/Corp/Enron@Enron\ncc: \nSubject: Credit Group Lunch\n\nA lunch meeting has been scheduled for Friday, May 5, 2000, from 12:00 p.m. \nuntil 1:30 p.m. in 30C2 to go through in detail how the ISDA and CSA Masters \nand Schedules work.\n\nPlease reply as soon as possible if you are going to attend this lunch \nmeeting (for catering purposes). Thanks.\n\n",0.358281,-3.020647
7,8,RE: Golf Anyone?,"Dasovich, Jeff JDASOVIC","[""'Nancy Sellers' <Nancy.Sellers@RobertMondavi.com""]",2001-10-17 08:50:02,"what stuff?\n\n-----Original Message-----\nFrom: Nancy Sellers [mailto:Nancy.Sellers@RobertMondavi.com]\nSent: Wednesday, October 17, 2001 10:50 AM\nTo: Dasovich, Jeff; psellers@pacbell.net; Nancy Sellers; eldon sellers\n(E-mail); Scott Laughlin (E-mail); cameron@perfect.com\nSubject: RE: Golf Anyone?\n\n\nShould I not mail this stuff then???\n\n-----Original Message-----\nFrom: Dasovich, Jeff [mailto:Jeff.Dasovich@ENRON.com]\nSent: Wednesday, October 17, 2001 8:27 AM\nTo: psellers@pacbell.net; Nancy Sellers (E-mail); eldon sellers\n(E-mail); Scott Laughlin (E-mail); cameron@perfect.com\nSubject: Golf Anyone?\n\n\nLooks like Prentice and Nancy will be getting together in Napa to do a\nlittle bonding this weekend. Therefore, it looks like an opportunity to\ngolf on Saturday or Sunday (screw football). Any takers? (Eldon has\npromised not to be the score Nazi.) \n\nBest,\nJeff\n\n\n**********************************************************

In [310]:
# raise RuntimeError("This is a purposeful error for demonstration.")

## The Generation Layer

In [311]:
# Define the function to generate the response. Provide a comprehensive prompt that passes the user query and the top 3 results to the model

def generate_response(user_query, top_5_results):
    """
    Generate a response using GPT's ChatCompletion based on the user query and retrieved information.
    """

    messages = [
        { "role": "system", "content": "You are a helpful assistant that provides accurate and concise answers based on the provided email content."},
        { "role": "user", "content": f"""
            You are a helpful assistant that provides accurate and concise answers based on the provided email content.
            You have a question asked by the user in '{user_query}' and you have top 5 emails in the dataframe '{top_5_results}'. 
         
            User Query: {user_query}

            Top 5 Relevant Emails:
            1. {top_5_results.iloc[0]['body']}
            2. {top_5_results.iloc[1]['body']}
            3. {top_5_results.iloc[2]['body']}
            4. {top_5_results.iloc[3]['body']}
            5. {top_5_results.iloc[4]['body']}

            The generated response should answer the query directly addressing the user and avoiding additional information. 
            If you think that the query is not relevant to the email content, reply that the query is irrelevant. 
            Provide the final response as a well-formatted and easily readable text along with the citation (subject, from, time). 
            Provide your complete response first with all information, and then provide the citations.
            """ }
    ]

    response = openai.chat.completions.create(
        model="gpt-4.1",
        messages=messages
    )

    return response.choices[0].message.content


In [312]:
# Generate the response

response = generate_response(query, top_5_emails_by_ranking)


In [313]:
# Print the response
print("Query:", query)
print("\nResponse:")
print(response)

Query: Who was part of Credit group lunch?

Response:
The following individuals were part of the Credit Group Lunch scheduled for May 5, 2000:

- Sara Shackleton
- Carol St Clair
- Mark Taylor
- Tana Jones
- Susan Flynn
- Susan Bailey
- Tanya Rohauer
- William S Bradford
- Debbie R Brackett
- Russell Diamond
- Veronica Espinoza
- Tracy Ngo
- Brant Reves
- Rod Nelson
- John Suttle
- Tom Moran
- Christopher Smith
- Lesli Campbell
- Mary Tudon (possibly referenced as Cathy Tudon)
- Paul Radous (possibly referenced as Kevin Radous)
- Molly Harris
- Nidia Mendoza (possibly referenced as Nidia Martinez)
- Edward Sacks
- Brad Schneider
- Wendi LeBrocq

Additionally, the following names are listed as "recently joined" and included:
- Ed Sacks
- Brad Schneider
- Wendy LeBrocq

Marie and Shari are specifically stated not to be included.

Citations:
- Subject: Credit Group Lunch, From: Carol St Clair, Time: 2000-04-18 04:54:00
- Subject: Credit Group Lunch, From: Suzanne Adams, Time: 2000-04-18 1